Importing DataSets and Libraries

In [ ]:
import pandas as pd
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split

In [ ]:
#Creating a base path to simplify our access to the files
base_path = pathlib.Path(r'C:\Users\Vitor Yago\Documents\GitHub\Data_Analysis\dataset\dataset')

#Creating the DataFrame
airbnb_base = pd.DataFrame()

#Creating a dictionary with the abbreviated name of the months (in Portuguese) 
months = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}

for file in base_path.iterdir():
    #Printing file name
    # print(file.name)

    #Year and Month
    month = months[file.name[:3]]
    year = file.name[-8:]
    year = year.replace('.csv', '')
    year = int(year)

    DataFrame = pd.read_csv(base_path / file.name)
    DataFrame['Year'] = year
    DataFrame['Month'] = month
    airbnb_base = airbnb_base.append(DataFrame)

Cleaning the base and creating a list with columns information.

After the cleanig, we only kept 34 columns

In [ ]:
#Creating a list with the columns name
columns = ['host_response_time','host_response_rate','host_is_superhost','host_listings_count','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','is_business_travel_ready','cancellation_policy','Year','Month']

#Filtering the Database
airbnb_base = airbnb_base.loc[:, columns]

#Checking if the filter worked
# print(list(airbnb_base.columns))
# display(airbnb_base)

Treating a Not a Number value (NaN)

In [ ]:
#How many columns have a NaN value?
print(airbnb_base.isnull().sum())

After analyzing the amount of NaN value, I decided to exclude the "reviews" columns from our dataset, because they have less then 50% of all data.

I'll exclude the columns with more than 300,000 NaN values

In [ ]:
for column in airbnb_base:
    if airbnb_base[column].isnull().sum() > 300000:
        airbnb_base = airbnb_base.drop(column, axis=1)

# print(airbnb_base.isnull().sum())

#For the rest of the columns that still have some NaN value, I'll exclude the line with the NaN value.
airbnb_base = airbnb_base.dropna()

print(airbnb_base.shape)
print(airbnb_base.isnull().sum())

Treating the data: What are the data types?

In [ ]:
#Comparing the type with the first information of every column
print(airbnb_base.dtypes)

In [ ]:
#price - need to be a float not object
airbnb_base['price'] = airbnb_base['price'].str.replace('$','')
airbnb_base['price'] = airbnb_base['price'].str.replace(',','')
airbnb_base['price'] = airbnb_base['price'].astype(np.float32, copy=False)

#extra people - need to be a float not object
airbnb_base['extra_people'] = airbnb_base['extra_people'].str.replace('$', '')
airbnb_base['extra_people'] = airbnb_base['extra_people'].str.replace(',', '')
airbnb_base['extra_people'] = airbnb_base['extra_people'].astype(np.float32, copy=False)

Treating the Outliers and finding the correlations between columns

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(airbnb_base.corr(), annot=True, cmap='Greens')

Functions area


In [ ]:
#Creating a function to determine the maximum and minimum of the analysis

def limits(column):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75)
    amp = q3 - q1
    return q1 - 1.5*amp, q3 + 1.5*amp

#Creating a function to do the box diagram
def box_diagram(column):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(15, 5)
    sns.boxplot(x=column, ax=ax1)
    ax2.set_xlim(limits(column))
    sns.boxplot(x=column, ax=ax2)

# def histogram(column):
#     plt.figure(figsize=(15, 5))
#     sns.distplot(column, hist=True)

#Creating a function to do the histogram
def histogram(column):
    plt.figure(figsize=(15, 5))
    sns.histplot(column)

#Creating a function to do a bar graph
def bar_graph(column):  
    plt.figure(figsize=(15, 5))
    ax = sns.barplot(x=column.value_counts().index, y=column.value_counts())
    ax.set_xlim(limits(column))

#Creating a function that excludes outliers
def exc_outliers(df, column_name):
    number_lines = df.shape[0]
    low_limit, upp_limit = limits(df[column_name])
    df = df.loc[(df[column_name] >= low_limit) | (df[column_name] <= upp_limit), :]
    deleted_lines = number_lines - df.shape[0]
    return df,  deleted_lines

In [ ]:
airbnb_base, deleted_lines = exc_outliers(airbnb_base, 'extra_people')
print('{} lines removed'.format(deleted_lines))

airbnb_base, deleted_lines = exc_outliers(airbnb_base, 'host_listings_count')
print('{} lines removed'.format(deleted_lines))

airbnb_base, deleted_lines = exc_outliers(airbnb_base, 'accommodates')
print('{} lines removed'.format(deleted_lines))

airbnb_base, deleted_lines = exc_outliers(airbnb_base, 'bathrooms')
print('{} lines removed'.format(deleted_lines))

airbnb_base, deleted_lines = exc_outliers(airbnb_base, 'bedrooms')
print('{} lines removed'.format(deleted_lines))

airbnb_base, deleted_lines = exc_outliers(airbnb_base, 'beds')
print('{} lines removed'.format(deleted_lines))

airbnb_base = airbnb_base.drop('guests_included', axis=1)
airbnb_base.shape

airbnb_base, deleted_lines = exc_outliers(airbnb_base, 'minimum_nights')
print('{} lines removed'.format(deleted_lines))

airbnb_base = airbnb_base.drop('maximum_nights', axis=1)
airbnb_base.shape

airbnb_base = airbnb_base.drop('number_of_reviews', axis=1)
airbnb_base.shape



In [ ]:
print(airbnb_base['property_type'].value_counts())

plt.figure(figsize=(15, 5))
graph = sns.countplot('property_type', data=airbnb_base)
graph.tick_params(axis='x', rotation=90)

In [ ]:
house_table_types = airbnb_base['property_type'].value_counts()
group_columns = []

for type in house_table_types.index:
    if house_table_types[type] < 2000:
        group_columns.append(type)
print(group_columns)

for type in group_columns:
    airbnb_base.loc[airbnb_base['property_type']==type, 'property_type'] = 'Other'

print(airbnb_base['property_type'].value_counts())
plt.figure(figsize=(15, 5))
grafico = sns.countplot('property_type', data=airbnb_base)
grafico.tick_params(axis='x', rotation=90)

In [ ]:
print(airbnb_base['cancellation_policy'].value_counts())

plt.figure(figsize=(15, 5))
graph = sns.countplot('cancellation_policy', data=airbnb_base)
graph.tick_params(axis='x', rotation=90)

table_cancellation = airbnb_base['cancellation_policy'].value_counts()
group_columns = []

for type in table_cancellation.index:
    if table_cancellation[type] < 10000:
        group_columns.append(type)
print(group_columns)

for type in group_columns:
    airbnb_base.loc[airbnb_base['cancellation_policy']==type, 'cancellation_policy'] = 'strict'

print(airbnb_base['cancellation_policy'].value_counts())
plt.figure(figsize=(15, 5))
graph = sns.countplot('cancellation_policy', data=airbnb_base)
graph.tick_params(axis='x', rotation=90)

In [ ]:
print(airbnb_base['amenities'].iloc[1].split(','))
print(len(airbnb_base['amenities'].iloc[1].split(',')))

airbnb_base['n_amenities'] = airbnb_base['amenities'].str.split(',').apply(len)
airbnb_base['n_amenities'] = airbnb_base['amenities'].str.split(',').apply(len)

airbnb_base = airbnb_base.drop('amenities', axis=1)
airbnb_base.shape

box_diagram(airbnb_base['n_amenities'])
bar_graph(airbnb_base['n_amenities'])

In [ ]:
sample = airbnb_base.sample(n=100000)
map_center = {'lat':sample.latitude.mean(), 'lon':sample.longitude.mean()}
map = px.density_mapbox(sample, lat='latitude', lon='longitude',z='price', radius=2.5,
                        center=map_center, zoom=10,
                        mapbox_style='stamen-terrain')
map.show()

In [ ]:
columns_tf = ['host_is_superhost', 'instant_bookable', 'is_business_travel_ready']
airbnb_base_cod = airbnb_base.copy()
for coluna in columns_tf:
    airbnb_base_cod.loc[airbnb_base_cod[coluna]=='t', coluna] = 1
    airbnb_base_cod.loc[airbnb_base_cod[coluna]=='f', coluna] = 0

categories_columns = ['property_type', 'room_type', 'bed_type', 'cancellation_policy']
airbnb_base_cod = pd.get_dummies(data=airbnb_base_cod, columns=categories_columns)
display(airbnb_base_cod.head())

In [ ]:
def evalueate_model(model_name, y_test, prevision):
    r2 = r2_score(y_test, prevision)
    RSME = np.sqrt(mean_squared_error(y_test, prevision))
    return f'Modelo {model_name}:\nR²:{r2:.2%}\nRSME:{RSME:.2f}'

In [ ]:
model_rf = RandomForestRegressor()
model_lr = LinearRegression()
model_et = ExtraTreesRegressor()

model = {'RandomForest': model_rf,
          'LinearRegression': model_lr,
          'ExtraTrees': model_et,
          }

y = airbnb_base_cod['price']
X = airbnb_base_cod.drop('price', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

for model_name, model in model.items():

    model.fit(X_train, y_train)

    forecast = model.predict(X_test)
    print(evalueate_model(model_name, y_test, forecast))

In [ ]:
for model_name, model in model.items():
    #testar
    predict = model.predict(X_test)
    print(evalueate_model(model_name, y_test, predict))

In [ ]:
#print(modelo_et.feature_importances_)
#print(X_train.columns)
feauture_importances = pd.DataFrame(model_et.feature_importances_, X_train.columns)
feauture_importances = feauture_importances.sort_values(by=0, ascending=False)
display(feauture_importances)
plt.figure(figsize=(15, 5))
ax = sns.barplot(x=feauture_importances.index, y=feauture_importances[0])
ax.tick_params(axis='x', rotation=90)

In [ ]:
X['price'] = y
X.to_csv('data.csv')
import joblib
joblib.dump(model_et, 'model.joblib')